In [1]:
# Import des librairies utilisées
import pandas as pd
import numpy as np
import openpyxl
import requests
import zipfile
import io
import tempfile
import os
import shutil
import subprocess
import sqlite3

---
**Récupération des données à partir d'URLs**
-
---

In [132]:
data_url = 'https://s3.eu-west-1.amazonaws.com/course.oc-static.com/projects/922_Data+Engineer/922_P4/Copie+de+Extraction+cube+OLAP+-+14+aout+2024.xlsx'
ventes = pd.read_excel(data_url, sheet_name='Vente Détail')
produits = pd.read_excel(data_url, sheet_name='Produits')
clients = pd.read_excel(data_url, sheet_name='Clients')
calendrier = pd.read_excel(data_url, sheet_name='Calendrier')
employes = pd.read_excel(data_url, sheet_name='Employé')
logs = pd.read_excel('https://s3.eu-west-1.amazonaws.com/course.oc-static.com/projects/922_Data+Engineer/922_P4/Logs.xlsx')
print('Ventes:')
print(ventes.head(5))
print('-------------')
print('Produits:')
print(produits.head(5))
print('-------------')
print('Clients:')
print(clients.head(5))
print('-------------')
print('Calendrier:')
print(calendrier.head(5))
print('-------------')
print('Employes:')
print(employes.head(5))
print('-------------')
print('Logs:')
print(logs.head(5))

Ventes:
                                ID_BDD        CUSTOMER_ID  \
0  HZDG8U15NNY7SI6HDK8NMFDEK7MOVUX31VY  CUST-G42Z6WE8QLWJ   
1  1H51BRR800TK9DCIH8M9QCRH3LEAR0JWALJ  CUST-CUA37GP8GABQ   
2  W6VNWK6ZX0D4QZ545ZD5Y0THWQQDBTXABXO  CUST-NHWI0DGNESYI   
3  58EC3JD1TDZZAVQZAW4AHFOUEKTR61ZLB70  CUST-IYXDMN7Q11O4   
4  96FE6HA91BRDDF6XRI3O9B83M5PKIXCIXZV  CUST-VR9O54A46EVF   

                         id_employe            EAN  Date achat ID ticket  
0  b413ca065a762e8cf2e86cfea8b9c174  6473630445822       45518    t_2693  
1  a7ada0770091e838e3dcd45265282820  1857802002765       45518    t_4408  
2  fa836e3f5faf72d24e079235332169ce  6831886714876       45518    t_3258  
3  6c1c3292c852c6c593b95cc146b00c0e  6916826750723       45518    t_1080  
4  38680e129f45c49322cb12303b7fb655  7930902861368       45518     t_979  
-------------
Produits:
             EAN                  categorie  Rayon            Libelle_produit  \
0  5026767366043  Produits Secs & Conserves  pates  500g penne rigate 

---
**Création de la base de données**
-
---

In [135]:
tmpdir = tempfile.TemporaryDirectory()
tmpdirname = tmpdir.name
db_path = os.path.join(tmpdirname, 'market.db')
proc = subprocess.run(["./create_db.sh", db_path], capture_output=True)
print(proc.stdout.decode())

Created /var/folders/78/_b7dcz_s2x9gjzb3rbnb4wbw0000gn/T/tmpdbkei_28/market.db database



---
**Import des données dans la base**
-
---

In [138]:
# Connexion à la base SQLITE
sql_conn = sqlite3.connect(db_path)
cur = sql_conn.cursor()

In [140]:
######## Import du calendrier
calendrier["Jour"] = calendrier["Jour"].apply(lambda x: str(x)[8:10])
calendrier.rename(columns={'Jour': 'jour', 'date': 'id_date'}, inplace=True)
calendrier.to_sql(
    name="calendrier",
    con=sql_conn,
    if_exists="append",
    index=False
)

1999

In [142]:
######## Import des clients
clients.rename(columns={'CUSTUMER_ID': 'id_client'}, inplace=True)
clients.to_sql(
    name="clients",
    con=sql_conn,
    if_exists="append",
    index=False
)

2297

In [144]:
######## Import des employés
employes.rename(columns={'employe': 'nom_utilisateur'}, inplace=True)
employes.to_sql(
    name="employes",
    con=sql_conn,
    if_exists="append",
    index=False
)

56

In [146]:
######## Import des produits
produits.rename(columns={'EAN': 'ean', 'Libelle_produit': 'libelle', 'Rayon': 'rayon'}, inplace=True)
produits.to_sql(
    name="produits",
    con=sql_conn,
    if_exists="append",
    index=False
)

18040

In [148]:
######## Import des ventes
ventes.rename(
    columns={
        'EAN': 'ean', 'ID_BDD': 'id_vente', 'CUSTOMER_ID': 'id_client',
        'Date achat': 'id_date', 'ID ticket': 'id_ticket'
    }
    , inplace=True)
ventes.to_sql(
    name="ventes",
    con=sql_conn,
    if_exists="append",
    index=False
)

41377

In [150]:
logs_kept = logs[
    ['id_user', 'date', 'action', 'table_insert', 'id_ligne', 'champs', 'detail']
    ].rename(
    columns={
        'table_insert': 'table_action', 'date': 'id_date'
    })
logs_kept.to_sql(
    name="logs",
    con=sql_conn,
    if_exists="append",
    index=False
)

207489

---
Q1: Chiffre d’affaires total pour les 14 Août et 15 août à partir des logs
-
---

In [171]:
cur.execute(
    '''
    WITH "Id_dates" AS (
        SELECT id_date as le_14
        FROM "Calendrier" WHERE annee=2024 AND mois=8 AND jour=14 
    ),
    "Transactions_date_valeur_le_14" AS (
        SELECT o.id_date
        FROM "Logs" o, "Id_dates" d
        WHERE o.action='INSERT' AND o.table_action='Ventes' AND o.champs='Date' AND CAST(o.detail AS INT)=d.le_14
        GROUP BY o.id_date
    ),
    "Chiffres_affaires" AS (
        SELECT date('1899-12-30', '+' || t.id_date || ' days') AS date, ROUND(CAST(SUM(p.prix) AS NUMERIC), 2) AS ca
        FROM "Logs" o, "Produits" p, "Transactions_date_valeur_le_14" t
        WHERE o.action='INSERT'  AND o.table_action='Ventes' AND o.champs='EAN' AND o.detail=p.ean AND o.id_date=t.id_date
        GROUP BY date
        ORDER BY date
    )
    SELECT CAST(date as text) AS "Date d'enregistrement", ca AS "Chiffre d’affaires enregistré pour le 14 août 2024"
    FROM "Chiffres_affaires"
    UNION ALL
    SELECT 'Total', SUM(ca)
    FROM "Chiffres_affaires"
    '''
)
res = cur.fetchall()
print('Réponse =>', res)

Réponse => [('2024-08-14', 275186.59), ('2024-08-15', 9057.29), ('Total', 284243.88)]


---
Q2: Chiffre d’affaires par client pour le top 10 des clients
-
---

In [174]:
cur.execute(
    '''
    SELECT v.id_client, ROUND(SUM(p.prix), 2) AS ca_par_client
    FROM calendrier c, ventes v, produits p
    WHERE c.id_date=v.id_date
                  AND v.ean=p.ean
                  AND c.annee=2024
                  AND c.jour=14
                  AND c.mois=8
    GROUP BY v.id_client
    ORDER BY ca_par_client DESC LIMIT 10
    '''
)
res = cur.fetchall()
print('Réponse =>', res)

Réponse => [('CUST-JNSOZSFORR88', 846.86), ('CUST-GM6VBAYAB8SF', 666.86), ('CUST-L2ST2JHI7K9O', 644.18), ('CUST-WU7ZKQJE4L17', 608.93), ('CUST-9WM83101QDTI', 582.03), ('CUST-ZMAOVX8XYGJY', 576.39), ('CUST-3K66CV0OHH7Q', 571.44), ('CUST-CG23SXJDRNYR', 531.09), ('CUST-D8IOFHVUFX3Y', 477.35), ('CUST-IHN1HQRI7PYJ', 463.73)]


---
Q3: Chiffre d’affaires encaissé par employé
-
---

In [179]:
cur.execute(
    '''
    WITH ca_par_employe AS (
                SELECT v.id_employe AS id_employe, ROUND(SUM(p.prix), 2) AS ca
                FROM calendrier c, ventes v, produits p
                WHERE c.id_date=v.id_date
                      AND v.ean=p.ean
                      AND c.annee=2024
                      AND c.jour=14
                      AND c.mois=8
                GROUP BY v.id_employe
                ORDER BY ca DESC
    ) 
    SELECT CONCAT(SUBSTR(e.prenom,1,1),'. ',e.nom), cpe.ca
    FROM employes e, ca_par_employe cpe
    WHERE e.id_employe=cpe.id_employe
    '''
)
res = cur.fetchall()
print('Réponse =>', res)

Réponse => [('A. Boulet', 7818.82), ('E. Jacquier', 7736.16), ('C. Delisle', 6995.14), ('P. Manoury', 6616.46), ('T. Arsenault', 6483.84), ('A. Dufresne', 6361.22), ('A. Dutertre', 6133.34), ('P. Ange', 6112.37), ('A. Lièvremont', 6111.18), ('P. Rodier', 6094.62), ('S. Deslys', 5989.84), ('C. Gachet', 5936.36), ('V. Baume', 5909.73), ('B. Cazal', 5827.75), ('I. Escoffier', 5734.83), ('É. Blanchard', 5705.28), ('U. Chevalier', 5567.78), ('S. Giraud', 5557.84), ('A. Donnet', 5513.15), ('S. Jacquet', 5490.3), ('S. Pélissier', 5490.26), ('J. Courbet', 5450.27), ('E. Grosjean', 5363.58), ('V. Genet', 5345.16), ('Y. Grinda', 5315.51), ('A. Marchal', 5220.68), ('M. Rochefort', 5187.41), ('G. Granet', 5187.01), ('J. Auch', 5146.52), ('A. Besson', 5128.45), ('A. Coquelin', 5013.37), ('L. Maret', 4941.48), ('V. Jacquier', 4775.66), ('R. Rémy', 4658.91), ('M. Poincaré', 4638.77), ('G. Bonhomme', 4621.61), ('C. Vérany', 4613.51), ('P. Beaumont', 4597.93), ('L. Boissonade', 4577.33), ('N. Picard', 

---
**Nettoyage**

---

In [130]:
# Fermeture de la connexion DB
sql_conn.close()
# Suppression du dossier temporaire
tmpdir.cleanup()